In [1]:
import numpy as np
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import names
import sys
import collections
import threading

sns.set(style="white")

In [2]:
time = 0
Lecture_time = 40            # in minutes
Scanning_time = 44 * 1000   # time for listening
Advertising_time = 16 * 1000      # time for advertising
Number_of_students = 10
Testing_times = 100         # loop times to get average performance
Classroom_height = 10         # size parameter
Classroom_width = 10         # size parameter
fail_count = 0
attend_duration_arr = []
time_out = 5 * 60 * 1000     # time out, maximum quiet for 5 minutes
Rx_Sensitivity = -90        # dBm, retrieved from test
SNR_threshold = 30           # dB, i.e. Rx = -40dBm and noise should <=-70dBm

In [3]:
class State:
    def __init__(self):
        self.SCANNING = "SCANNING"
        self.ADVERTISING = "ADVERTISING"
        self.INACTIVATED = "INACTIVATED"
state = State()

In [4]:
def list_xor(list1, list2):
    ans = 0
    for i in range(len(list1)):
        ans |= list1[i] ^ list2[i]
    return ans

def list_or(list1, list2):
    res = [0] * len(list1)
    for i in range(len(list1)):
        res[i] = list1[i] | list2[i]
    return res

In [5]:
class Teacher:
    def __init__(self, x = None, y = None):
        self.x = x
        self.y = y
        self.identity = 'Teacher'
        self.cur_state = state.SCANNING
        self.Rx_Power = 0
        self.Noise_Power = 0
        self.SNR = 0
        self.bit_map = [0] * Number_of_students
    
    def co_channel_noise(self, students, index_of_this_stu):
        n = 0
        sum = 0
        for s in students:
            if s.cur_state == state.ADVERTISING and s.stu_index != index_of_this_stu:
                n = pathloss_model(s.Tx_Power, cal_dis(self.x,self.y,s.x,s.y))   # in dBm
                sum += 10**(n/10.0)    # in milliwatt
        if sum==0:
            return 0                       # in dBm
        else:
            return 10 * math.log10(sum)    # in dBm
            
    def listen(self, students):
        if self.cur_state == state.SCANNING:
            for s in students:
                if s.cur_state == state.ADVERTISING and s.mask:
                    self.Rx_Power = pathloss_model(s.Tx_Power, cal_dis(self.x,self.y,s.x,s.y))
                    # within range
                    if (self.Rx_Power >= Rx_Sensitivity):
                        self.Noise_Power = gaussian_noise() + self.co_channel_noise(students, s.stu_index)
                        self.SNR = self.Rx_Power - self.Noise_Power
                        # signal to noise ratio larger than required
                        if(self.SNR >= SNR_threshold and np.random.rand()<0.5):
                            # message received!
                            if (list_xor(self.bit_map, s.bit_map)):
                                # include the changes into my list
                                self.bit_map = list_or(self.bit_map, s.bit_map)
        else:
            # inactivated or advertising
            return

In [6]:
class Seat:
    def __init__(self, x = None, y = None, proba = 0, anc = None, ident = 'Empty'):
        self.x = x
        self.y = y
        self.identity = ident # identity: Empty, Student
        self.cur_state = state.INACTIVATED # SCANNING, ADVERTISING
        # let's say all students comes in LT within 1 minute
        self.start_time = np.random.randint(60*1000)
        self.bias = time_bias()
        self.mask = True
        self.bit_map = [0] * Number_of_students
        self.stu_index = 0
        self.Tx_Power = 10-30*np.random.random()   # -20~+10 dBm
        self.Rx_Power = 0
        self.Noise_Power = 0
        self.SNR = 0
#         self.rest = False
        self.rest_start_time = 0
    
    def set_x(self, x):
        self.x = x

    def set_y(self, y):
        self.y = y
        
    def set_identity(self, ident):
        self.identity = ident 
    
    def set_bitmap(self, index):
        self.bit_map[index] = 1

    def co_channel_noise(self, students, index_of_this_stu):
        n = 0
        sum = 0
        for s in students:
            if s.cur_state == state.ADVERTISING and s.stu_index != index_of_this_stu:
                n = pathloss_model(s.Tx_Power, cal_dis(self.x,self.y,s.x,s.y))   # in dBm
                sum += 10**(n/10.0)    # in milliwatt
        if sum==0:
            return 0                       # in dBm
        else:
            return 10 * math.log10(sum)    # in dBm

    def listen(self, students, teacher, t_x, t_y):
        # if self is scanning
        if self.cur_state == state.SCANNING:
            for s in students:
                # if student s is advertising and first time advertising in a while
                if s.cur_state == state.ADVERTISING and s.mask:
                    self.Rx_Power = pathloss_model(s.Tx_Power, cal_dis(self.x,self.y,s.x,s.y))
                    # within range
                    if (self.Rx_Power >= Rx_Sensitivity):
                        self.Noise_Power = gaussian_noise() + self.co_channel_noise(students, s.stu_index)
                        self.SNR = self.Rx_Power - self.Noise_Power
                        # signal to noise ratio larger than required
                        if(self.SNR >= SNR_threshold and np.random.rand()<0.5):
                            # message received!
                            if (list_xor(self.bit_map, s.bit_map)):
                                # include the changes into my list
                                self.bit_map = list_or(self.bit_map, s.bit_map)
                                self.rest = False
                            else:
                                # let the device rest and listen only
                                if not self.rest: 
                                    # if this is first time to rest
                                    self.rest_start_time = time
                                self.rest = True
        else:
            # inactivated or advertising
            return

    def update_state(self):
        # if current time hasn't hit start time, keep this student inactivated
        if time < self.start_time:
            self.cur_state = state.INACTIVATED
        # in last period of scan, there was difference in bit_map
#         elif not self.rest:
        else:
            mod = (time - self.start_time) % (Advertising_time + Scanning_time)
            # 0 ~ Advertising_time
            if mod < self.bias:
                self.cur_state = state.INACTIVATED
            elif mod < Advertising_time:
                # Advertising one data packet every 1.6s, each packet lasts 6ms
                if mod%(1.8*1000) <= 6:
                    if mod%(1.8*1000) == 0 and (not self.mask):
                        # make it broadcast once within one packet's time
                        self.mask = True
                    elif self.mask:
                        self.mask = False
                    # avoid repeated assignment here
                    if self.cur_state != state.ADVERTISING:
                        self.cur_state = state.ADVERTISING
                # inactivate it in intervals
                elif self.cur_state != state.INACTIVATED:
                    self.cur_state = state.INACTIVATED
            # Advertising_time ~ end of period
            elif self.cur_state != state.SCANNING:
                self.cur_state = state.SCANNING
                # determine next period's bias
                self.bias = time_bias()
        # rests until it scans a difference in bit_map
        elif self.rest:
            if self.cur_state != state.SCANNING:
                self.cur_state = state.SCANNING
            # if the student rests for more than 5 min
            if(time - self.rest_start_time >= time_out):
                self.rest = False
                self.rest_start_time = time

In [7]:
def cal_dis(x1,y1,x2,y2):
    return math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))

def pathloss_model(Tx_Power, d):
    # (dBm) = (dBm) - (dB)
    if d>0.05:
        return Tx_Power - (20*math.log10(d) + 40.05)
    else:
        return Tx_Power
    
def gaussian_noise():
    # noise with mean -70 dBm and variance 20 dB
    return np.random.normal(-50, 30)

def time_bias():
    return np.random.randint(10)

In [8]:
class Classroom:

    def __init__(self, length = -1, width= -1):
        self.seat_length = length
        self.seat_width = width
        #list where all the student names are
        seat_list = []
        for l in range(length):
            for w in range(width):
                seat_list.append(Seat(w,l, ident="Empty"))
        self.class_seats = seat_list
        self.teacher_x = self.seat_width / 2
        self.teacher_y = -1
        self.teacher = Teacher(self.teacher_x,self.teacher_y)
        self.fail_indicator = False
        
    def get_seat(self, x, y):
        return self.class_seats[x + (y * self.seat_width)] # return the Seat OBJECT
        
    def convert_index_to_xy(self,index):
        x = index % self.seat_width
        y = math.floor(index / self.seat_width)
        return x, y
    
    def convert_xy_to_index(self, x, y):
        return x + (y * self.seat_width)
    
    def random_arrange_student(self, student_num):
        """
        Randomly arrange students for this class
        student_number: Number of students in this class
        """
        if len([self.return_all_student_seat()]) > 0:
            # If there are already students in this class
            # Reset all the seats in this class
            self.class_seats = []
            for l in range(self.seat_length):
                for w in range(self.seat_width):
                    self.class_seats.append(Seat(w,l, ident="Empty"))
        arr = np.arange(self.seat_length * self.seat_width)
        np.random.shuffle(arr)
        np.random.shuffle(arr)
        
        for index in arr[:student_num]:
            self.class_seats[index].set_identity("Student")
        
        t = 0
        for x in range(Classroom_width):
            for y in range(Classroom_height):
                if(self.get_seat(x,y).identity == "Student"):
                    self.get_seat(x,y).stu_index = t
                    self.get_seat(x,y).set_bitmap(t)
                    t += 1

    def return_all_student_seat(self):
        res = [] 
        for i in self.class_seats:
            if (i.identity == "Student"):
                res.append(i)
        # res = [Seat1_object, Seat2_object, Seat3_object,....]
        return res
    
    def visualize_students(self):
        """
        Visualize all the student seats in this class
        """
        students = self.return_all_student_seat()
        arr = np.array([[-1] * self.seat_width] * self.seat_length) # Initializa a 2d-array with class_width * class_length, initial value is 0
        for s in students:
            arr[s.y][s.x] = s.stu_index
            
        # Draw the heatmap
        plt.figure(figsize=(self.seat_length, self.seat_width))
        sns.heatmap(pd.DataFrame(arr).sort_index(ascending=False), 
                    square=True, 
                    linewidths=0.5, 
                    cmap = "YlGnBu",
                    cbar = False,
                    annot = True)
        plt.savefig("./stu_location.png")
        plt.show()
        return arr
    
    def visualize_selected_seats(self, seats_list):
        arr = np.array([[0] * self.seat_width] * self.seat_length)
        for s in seats_list:
            arr[s.y][s.x] = 1
        
        plt.figure(figsize=(self.seat_length/2, self.seat_width/2))
        sns.heatmap(pd.DataFrame(arr).sort_index(ascending=False), 
                    square=True, 
                    linewidths=0.5, 
                    cmap = "YlGnBu",
                    cbar = False,
                    annot = True)
        plt.show()
        return arr
    
    def all_here(self):
        return collections.Counter(self.teacher.bit_map) == collections.Counter([1]*Number_of_students)
    
    def loop_once(self):
        students = self.return_all_student_seat()
        #sampling rate is 1ms
        global time
        time += 1
        for s in students:
            s.update_state()
            s.listen(students,self.teacher,self.teacher_x, self.teacher_y)
            self.teacher.listen(students)

In [11]:
# with open("test_data_v4_500.txt", 'a') as out:
#     out.write("Period: "+str((Advertising_time+Scanning_time)/1000)+" s. " + "Probability of success: 0.5. " + "Stu Num: " + str(Number_of_students) + ". Size(h*w): " + str(Classroom_height) + '*' + str(Classroom_width) + ". Ad+Sc ratio: " + str(Advertising_time/1000) + '+' + str(Scanning_time/1000) + '\n')
for j in range(Testing_times):
    global time
    global fail_count
    time = 0
#     showit = False
    print("loop: " + str(j))
    A = Classroom(Classroom_height,Classroom_width)
    A.random_arrange_student(Number_of_students)
    while(not A.all_here()):
        A.loop_once()
        if(time >= 60*1000*Lecture_time):  # if the time exceeds 45 mins, stop the simulation
            A.fail_indicator = True
            break
#         if(time/1000>=300):
#             if(time%1000==0):
#                 print("At time(min) " + str(time/(60*1000)) + str(A.teacher.bit_map))
#             if(not showit):
#                 A.visualize_students()
#                 showit = True
#     print("Finally " + str(A.teacher.bit_map))
    # to ensure failure is not included
    if (not A.fail_indicator):    
        attend_duration_arr.append(time*1.0/1000)
    else:
        fail_count += 1
    print("=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=")
#     with open("test_data_v4_500.txt", 'a') as out:
#         out.write(str(time*1.0/1000) + '\n')

loop: 0
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 1
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 2
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 3
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 4
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 5
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 6
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 7
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 8
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=

Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 70
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 71
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 72
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 73
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 74
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 75
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 76
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
loop: 77
Finally [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
=

In [ ]:
print("\n\nAverage time taken is " + str(round(np.mean(attend_duration_arr),2)))
print("Standard deviation is " + str(round(np.std(attend_duration_arr),4)))
print("90 Percentile is " + str(np.percentile(attend_duration_arr, 90)))
print("Failure = " + str(fail_count) + " out of " + str(Testing_times) + " times of trial")
plt.scatter(range(len(attend_duration_arr)), np.array(attend_duration_arr))
plt.show()